In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
%cd /gdrive/MyDrive/Code\ Repository/aidea/ANS/AIdea_ANS

/gdrive/MyDrive/Code Repository/aidea/ANS/AIdea_ANS


In [ ]:
!pip install -r requirements.txt

In [ ]:
from torch.cuda import is_available
print(is_available())

False


In [17]:
!python train.py

{'train_data_path': '/gdrive/MyDrive/Code Repository/aidea/ANS/data/train_classification', 'test_data_path': '/gdrive/MyDrive/Code Repository/aidea/ANS/data/test', 'saving_path': '/gdrive/MyDrive/Code Repository/aidea/ANS/data', 'class': 'train', 'train_per': 0.1, 'eposide': 5, 'signal_bound': 185000, 'epochs': 1, 'batch_size': 4, 'lr': 0.0002, 'beta1': 0.5, 'beta2': 0.999, 'gamma': 0.99}
train_num:	 166
device:	 cpu
version:	 11
epoch:	 0
  0% 0/42 [00:00<?, ?it/s]

	 (4, 1, 100000)
(1)	 0.05180478096008301
(2)	 15.213671207427979
(3)	 0.049466609954833984
(4)	 4.293179273605347
(5)	 23.737128496170044
  2% 1/42 [01:00<41:30, 60.75s/it]

	 (4, 1, 100000)
(1)	 0.019537687301635742
(2)	 11.193772554397583
(3)	 0.051381826400756836
(4)	 3.4930529594421387
^C


In [9]:
!python train.py

{'train_data_path': '/gdrive/MyDrive/Code Repository/aidea/ANS/data/train_classification', 'test_data_path': '/gdrive/MyDrive/Code Repository/aidea/ANS/data/test', 'saving_path': '/gdrive/MyDrive/Code Repository/aidea/ANS/data', 'class': 'train', 'train_per': 0.1, 'eposide': 5, 'signal_bound': 185000, 'epochs': 1, 'batch_size': 2, 'lr': 0.0002, 'beta1': 0.5, 'beta2': 0.999, 'gamma': 0.99}
train_num:	 166
device:	 cpu
version:	 6
epoch:	 0
  0% 0/83 [00:00<?, ?it/s](1)	 0.03446769714355469
(2))	 35.2728590965271
(3)	 0.18766236305236816
(4)	 5.224318742752075
(5)	 76.61475157737732
  1% 1/83 [01:57<2:40:52, 117.71s/it](1)	 0.04028487205505371
  1% 1/83 [02:12<3:01:36, 132.89s/it]
Traceback (most recent call last):
  File "train.py", line 140, in <module>
    main()
  File "train.py", line 83, in main
    torch.from_numpy(ori_f_3).float()))
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File